In [1]:
!pip install llama-index llama-parse llama-index-embeddings-huggingface llama-index-llms-huggingface

In [2]:
# Read PDF using llama-parse
from llama_parse import LlamaParse
import nest_asyncio
import os
from google.colab import userdata

os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get('LLAMA_CLOUD') #LLAMA CLOUD API
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_1') #OPENAI_API_KEY_1
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
nest_asyncio.apply()
pdf_file_name = "/content/Final+Policy+document_LICs+New+Jeevan+Shanti_V05_logo.pdf"

# Download LIC JEEVAN SHANTI PDF

In [3]:
!wget https://licindia.in/documents/20121/62876/Final+Policy+document_LICs+New+Jeevan+Shanti_V05_logo.pdf

--2024-08-03 18:23:10--  https://licindia.in/documents/20121/62876/Final+Policy+document_LICs+New+Jeevan+Shanti_V05_logo.pdf
Resolving licindia.in (licindia.in)... 64.185.181.238
Connecting to licindia.in (licindia.in)|64.185.181.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 588928 (575K) [application/pdf]
Saving to: ‘Final+Policy+document_LICs+New+Jeevan+Shanti_V05_logo.pdf.2’

Final+Policy+docume 100%[===================>] 575.12K  --.-KB/s    in 0.007s  

2024-08-03 18:23:10 (81.3 MB/s) - ‘Final+Policy+document_LICs+New+Jeevan+Shanti_V05_logo.pdf.2’ saved [588928/588928]



# Read PDF

In [4]:
documents = LlamaParse(result_type="markdown").load_data(pdf_file_name)
print(documents[0].text[:1000])

Started parsing the file under job_id 733115c2-fcf7-426d-b851-beda3c73d8bc
# LIFE INSURANCE CORPORATION OF INDIA

(Established by the Life Insurance Corporation Act, 1956)

Registration Number: 512

# LIC’S NEW JEEVAN SHANTI (UIN: 512N338V05)

(A Non-Linked, Non-Participating, Individual, Single Premium, Deferred Annuity Plan)

# PART – A

Ref: NB (Address and e-mail id of Branch Office):

Space for Name and Address of Policyholder
Space for Address and e-mail id of Branch Office

Dear Policyholder,

Date: _______________

Re: Your Policy No. _______________

We have pleasure in forwarding herewith the above policy document comprising of Part A to Part G which please find in order.

We would also like to draw your kind attention to the information mentioned in the Schedule of the Policy and the benefits available under the Policy.

Some of our Plans have certain options available under them. It is important that the options, if any, available under this Plan and mentioned in the Policy

# Embedding models

In [5]:
#Open-AI embeddings
from llama_index.embeddings.openai import OpenAIEmbedding
openai_emb = OpenAIEmbedding()

#bge embedding from Huggingface
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
bge_emb = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

#jinaai embeddings from huggingface
jinaai_emb = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-base-en")

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

# LLMs


In [6]:
from llama_index.llms.openai import OpenAI
gpt_4_llm = OpenAI(model="gpt-4", temperature=0.1)

from llama_index.llms.huggingface import HuggingFaceLLM
mistral_llm = HuggingFaceLLM(model_name="mistralai/Mistral-7B-Instruct-v0.1")

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Vector Store Index

In [7]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents, embed_model=openai_emb, llm=gpt_4_llm)
openai_gpt4_engine = index.as_query_engine(llm=gpt_4_llm)
print(openai_gpt4_engine.query("What is Free-look period?").response)
# index_bge = VectorStoreIndex.from_documents(documents, embed_model=bge_emb)
# index_jinaai = VectorStoreIndex.from_documents(documents, embed_model=jinaai_emb)

The Free Look period is a span of 30 days from the date of receipt of the policy document, either in electronic or physical form, whichever is earlier. During this period, if the policyholder is not satisfied with the terms and conditions of the policy, they may return the policy to the Corporation stating their objections. Upon receipt, the Corporation will cancel the policy and return the premium paid after deducting charges for stamp duty and any annuity paid. This condition is only applicable in the case of a new purchase of a Deferred Annuity plan. If the purchase is from an existing fund, this condition does not apply.


In [8]:
index = VectorStoreIndex.from_documents(documents, embed_model=bge_emb, llm=gpt_4_llm)
bge_gpt4_engine = index.as_query_engine(llm=gpt_4_llm)
print(bge_gpt4_engine.query("What is Free-look period?"))

The Free Look period is a span of 30 days from the date of receipt of the electronic or physical mode of the Policy Document, whichever comes first, by the Policyholder. During this period, if the Policyholder is not satisfied with the Terms and Conditions of the policy, they may return the policy to the Corporation stating their objections. Upon receipt, the Corporation will cancel the Policy and return the Premium paid after deducting the charges for stamp duty and annuity paid, if any. This condition of Free Look period is only applicable in case of new purchase of Deferred Annuity plan. If the purchase is from the existing fund, this condition is not applicable.


In [9]:
index = VectorStoreIndex.from_documents(documents, embed_model=jinaai_emb, llm=gpt_4_llm)
jin_gpt4_engine = index.as_query_engine(llm=gpt_4_llm)
print(jin_gpt4_engine.query("What is Free-look period?"))

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


The Free Look Period is a span of 30 days from the date of receipt of the policy document, either in electronic or physical mode, whichever is earlier. During this period, the policyholder can review the terms and conditions of the policy. If the policyholder disagrees with any of the terms and conditions, they have the option to return the policy.


In [10]:
index = VectorStoreIndex.from_documents(documents, embed_model=openai_emb, llm=mistral_llm)
openai_mistral_engine = index.as_query_engine()
print(openai_mistral_engine.query("What is Free-look period?"))

The Free Look period allows the Policyholder to return the policy to the Corporation within 30 days from the date of receiving the electronic or physical mode of the Policy Document if they are not satisfied with the Terms and Conditions of the policy. Upon returning the policy during this period, the Corporation will cancel the policy and return the Premium paid after deducting charges for stamp duty and annuity paid, if any.


In [11]:
index = VectorStoreIndex.from_documents(documents, embed_model=bge_emb, llm=mistral_llm)
bge_mistral_engine = index.as_query_engine()
print(bge_mistral_engine.query("What is Free-look period?"))

The Free Look period allows the Policyholder to return the policy to the Corporation within 30 days from the date of receiving the electronic or physical mode of the Policy Document if they are not satisfied with the Terms and Conditions of the policy. Upon returning the policy during this period, the Corporation will cancel the policy and return the Premium paid after deducting the charges for stamp duty and annuity paid, if any.


In [12]:
index = VectorStoreIndex.from_documents(documents, embed_model=jinaai_emb, llm=mistral_llm)
jin_mistral_engine = index.as_query_engine()
print(jin_mistral_engine.query("What is Free-look period?"))

The Free Look Period is a period of 30 days from the date of receipt of the electronic or physical mode of the Policy Document, whichever is earlier, by the Policyholder to review the terms and conditions of the policy. During this period, if the Policyholder disagrees with any of the terms and conditions, they have the option to return the policy as detailed in the policy document.


In [13]:

from llama_index.core.llama_pack import download_llama_pack

RagEvaluatorPack = download_llama_pack(
  "RagEvaluatorPack", "./rag_evaluator_pack"
)

In [14]:
from llama_index.core import ServiceContext
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

service_context_gpt4 = ServiceContext.from_defaults(llm=gpt_4_llm)

dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    service_context=service_context_gpt4,
    num_questions_per_chunk=2,
    show_progress=True,
)

eval_dataset = dataset_generator.generate_dataset_from_nodes()

<ipython-input-14-18270d0f20a1>:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context_gpt4 = ServiceContext.from_defaults(llm=gpt_4_llm)


Parsing nodes:   0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:03<00:00,  1.53s/it]


In [15]:
from llama_index.packs.rag_evaluator import RagEvaluatorPack
import pandas as pd
async def evaluate(query_engine, qe_st):
  rag_evaluator_pack = RagEvaluatorPack(
      rag_dataset=eval_dataset,
      query_engine=query_engine
  )

  benchmark_df = await rag_evaluator_pack.arun(
      batch_size=10,  # batches the number of openai api calls to make
      sleep_time_in_seconds=1,  # seconds to sleep before making an api call
  )
  return benchmark_df
openai_gpt4_engine_metrics = await evaluate(openai_gpt4_engine, "OpenAI GPT-4")
bge_gpt4_engine_metrics = await evaluate(bge_gpt4_engine, "BGE GPT-4")
jin_gpt4_engine_metrics = await evaluate(jin_gpt4_engine, "JINA AI GPT-4")
openai_mistral_engine_metrics = await evaluate(openai_mistral_engine, "OpenAI Mistral")
bge_mistral_engine_metrics = await evaluate(bge_mistral_engine, "BGE Mistral")
jin_mistral_engine_metrics = await evaluate(jin_mistral_engine, "JINA AI Mistral")

Batch processing of predictions: 100%|██████████| 2/2 [00:04<00:00,  2.36s/it]
Batch processing of evaluations:  93%|█████████▎| 21/22.5 [02:01<00:08,  5.77s/it]
Batch processing of predictions: 100%|██████████| 2/2 [00:04<00:00,  2.39s/it]
Batch processing of evaluations:  93%|█████████▎| 21/22.5 [01:56<00:08,  5.56s/it]
Batch processing of predictions: 100%|██████████| 2/2 [00:06<00:00,  3.38s/it]
Batch processing of evaluations:  93%|█████████▎| 21/22.5 [01:53<00:08,  5.42s/it]
Batch processing of predictions: 100%|██████████| 2/2 [00:02<00:00,  1.41s/it]
Batch processing of evaluations:  93%|█████████▎| 21/22.5 [02:06<00:09,  6.03s/it]
Batch processing of predictions: 100%|██████████| 2/2 [00:02<00:00,  1.09s/it]
Batch processing of evaluations:  93%|█████████▎| 21/22.5 [02:03<00:08,  5.88s/it]
Batch processing of predictions: 100%|██████████| 2/2 [00:02<00:00,  1.06s/it]
Batch processing of evaluations:  93%|█████████▎| 21/22.5 [02:05<00:08,  5.97s/it]


In [19]:
final_results = pd.concat([openai_gpt4_engine_metrics, bge_gpt4_engine_metrics, jin_gpt4_engine_metrics, openai_mistral_engine_metrics, bge_mistral_engine_metrics, jin_mistral_engine_metrics], axis=1)
final_results.columns = ['OpenAI GPT-4', 'BGE GPT-4', 'JINA AI GPT-4', 'OpenAI Mistral', 'BGE Mistral', 'JINA AI Mistral']
final_results

,OpenAI GPT-4,BGE GPT-4,JINA AI GPT-4,OpenAI Mistral,BGE Mistral,JINA AI Mistral
metrics,,,,,,
mean_correctness_score,4.488095,4.392857,2.071429,4.345238,4.238095,3.166667
mean_relevancy_score,0.880952,0.833333,0.357143,0.952381,0.880952,0.523810
mean_faithfulness_score,0.952381,0.928571,0.714286,0.976190,0.952381,0.761905
mean_context_similarity_score,0.961225,0.959758,0.883087,0.961217,0.959767,0.883096


**Results show great performance with OpenAI Embeddings, and Mistral, GPT-4 as LLMs.**